In [93]:
import os
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight

np.random.seed(42)
tf.random.set_seed(42)

DATA_PATH = os.path.join(os.getcwd(), 'data', 'breast-cancer-wisconsin.data')

In [94]:
names = [
    'id', 
    'clump_thickness', 
    'cell_size_uniformity', 
    'cell_shape_uniformity',
    'marginal_adhesion',
    'epithelial_cell_size',
    'bare_nuclei',
    'bland_chromatin',
    'normal_nucleoli',
    'mitoses',
    'class'
]

df = pd.read_csv(DATA_PATH, names=names)

# remove NaN values
df = df.replace('?', np.NaN)
df.dropna(inplace=True)

# remove non-feature col
df.drop(columns=['id'], inplace=True)

# convert labels to M=1 and B=0
df['class'] = (df['class'] == 4).astype(int)

df.head()

,clump_thickness,cell_size_uniformity,cell_shape_uniformity,marginal_adhesion,epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,5,1,1,1,2,1,3,1,1,0
1,5,4,4,5,7,10,3,2,1,0
2,3,1,1,1,2,2,3,1,1,0
3,6,8,8,1,3,4,3,7,1,0
4,4,1,1,3,2,1,3,1,1,0


In [95]:
# spilt features and labels
X = df.drop(columns=['class'])
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [96]:
# scale features between 0 and 1
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [97]:
# compute class weights (data is unbalanced)
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = {
    0: class_weights[0],
    1: class_weights[1],
}
class_weights

{0: 0.7479452054794521, 1: 1.5082872928176796}

In [112]:
# model architecture
model = keras.models.Sequential([
    keras.layers.Input(shape=[9,]),
    keras.layers.Dense(16, activation='selu'),
    keras.layers.Dense(8, activation='selu'),
    keras.layers.Dense(1, activation='sigmoid'),
])

early_stop = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [113]:
model.fit(X_train, y_train, epochs=50, 
          validation_data=(X_test, y_test), 
          class_weight=class_weights,
          callbacks=[early_stop])

Epoch 1/50
18/18 [==============================] - 0s 6ms/step - loss: 0.6533 - accuracy: 0.4469 - val_loss: 0.5620 - val_accuracy: 0.6423
Epoch 2/50
18/18 [==============================] - 0s 2ms/step - loss: 0.5221 - accuracy: 0.6154 - val_loss: 0.4797 - val_accuracy: 0.7518
Epoch 3/50
18/18 [==============================] - 0s 2ms/step - loss: 0.4363 - accuracy: 0.7729 - val_loss: 0.4126 - val_accuracy: 0.8467
Epoch 4/50
18/18 [==============================] - 0s 2ms/step - loss: 0.3677 - accuracy: 0.8938 - val_loss: 0.3441 - val_accuracy: 0.9489
Epoch 5/50
18/18 [==============================] - 0s 2ms/step - loss: 0.3059 - accuracy: 0.9524 - val_loss: 0.2866 - val_accuracy: 0.9635
Epoch 6/50
18/18 [==============================] - 0s 2ms/step - loss: 0.2517 - accuracy: 0.9560 - val_loss: 0.2364 - val_accuracy: 0.9708
Epoch 7/50
18/18 [==============================] - 0s 2ms/step - loss: 0.2061 - accuracy: 0.9634 - val_loss: 0.1939 - val_accuracy: 0.9708
Epoch 8/50
18/18 [==